In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import math
import json

In [2]:
# load dataset
df = pd.read_csv("datasets/airline_delay_causes/airline_delay_causes.csv")

In [4]:
df.info();

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42894 entries, 0 to 42893
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   year                 42894 non-null  int64  
 1    month               42894 non-null  int64  
 2   carrier              42894 non-null  object 
 3   carrier_name         42894 non-null  object 
 4   airport              42894 non-null  object 
 5   airport_name         42894 non-null  object 
 6   arr_flights          42788 non-null  float64
 7   arr_del15            42612 non-null  float64
 8   carrier_ct           42788 non-null  float64
 9    weather_ct          42788 non-null  float64
 10  nas_ct               42788 non-null  float64
 11  security_ct          42788 non-null  float64
 12  late_aircraft_ct     42788 non-null  float64
 13  arr_cancelled        42788 non-null  float64
 14  arr_diverted         42788 non-null  float64
 15   arr_delay           42788 non-null 

In [5]:
df.describe()

,year,month,arr_flights,arr_del15,carrier_ct,weather_ct,nas_ct,security_ct,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,Unnamed: 21
count,42894.000000,42894.000000,42788.000000,42612.000000,42788.000000,42788.000000,42788.000000,42788.000000,42788.000000,42788.000000,42788.000000,42788.000000,42788.000000,42788.000000,42788.000000,42788.000000,42788.000000,0.0
mean,2019.583695,6.068005,299.249486,44.628931,13.410486,1.668595,14.339095,0.105468,14.921771,10.237356,0.650907,2993.014513,997.864822,184.684187,703.981350,4.923694,1101.548565,NaN
std,0.622120,3.596389,866.798596,137.501141,35.897800,5.809364,53.038616,0.530672,51.336074,71.506755,3.057961,10229.656136,3172.507743,762.479517,3168.981732,33.077606,3977.949252,NaN
min,2019.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
25%,2019.000000,3.000000,33.000000,4.000000,1.000000,0.000000,0.500000,0.000000,0.260000,0.000000,0.000000,169.000000,51.000000,0.000000,16.000000,0.000000,9.000000,NaN
50%,2020.000000,6.000000,82.000000,11.000000,3.970000,0.010000,2.580000,0.000000,2.700000,1.000000,0.000000,621.000000,221.000000,1.000000,93.000000,0.000000,165.000000,NaN
75%,2020.000000,9.000000,197.000000,30.000000,10.690000,1.280000,7.970000,0.000000,9.240000,4.000000,0.000000,1891.250000,721.000000,103.000000,319.000000,0.000000,661.000000,NaN
max,2021.000000,12.000000,21873.000000,3714.000000,966.330000,175.560000,1485.820000,26.070000,1429.550000,4951.000000,115.000000,329847.000000,97970.000000,23749.000000,97283.000000,3194.000000,125086.000000,NaN


## All columns and their description

1. Year: ~~ 1987-2008
2. Month: ~~ 1-12
3. DayofMonth: ~~ 1-31
4. DayOfWeek: ~~ 1 (Monday) - 7 (Sunday)
5. DepTime: ~~ actual departure time (local, hhmm)
6. CRSDepTime: ~~ scheduled departure time (local, hhmm)
7. ArrTime: ~~ actual arrival time (local, hhmm)
8. CRSArrTime: ~~ scheduled arrival time (local, hhmm)
9. UniqueCarrier: ~~ unique carrier code
10. FlightNum: ~~ flight number
11. TailNum: ~~ plane tail number
12. ActualElapsedTime: ~~ in minutes
13. CRSElapsedTime: ~~ in minutes
14. AirTime: ~~ in minutes
15. ArrDelay: ~~ arrival delay, in minutes
16. DepDelay: ~~ departure delay, in minutes
17. Origin: ~~ origin IATA airport code
18. Dest: ~~ destination IATA airport code
19. Distance: ~~ in miles
20. TaxiIn: ~~ taxi in time, in minutes
21. TaxiOut: ~~ taxi out time in minutes
22. Cancelled: ~~ was the flight cancelled?
23. CancellationCode: ~~ reason for cancellation (A = carrier, B = weather, C = NAS, D = security)
24. Diverted: ~~ 1 = yes, 0 = no
25. CarrierDelay: ~~ in minutes
26. WeatherDelay: ~~ in minutes
27. NASDelay: ~~ in minutes
28. SecurityDelay: ~~ in minutes
29. LateAircraftDelay: ~~ in minutes

In [6]:
df.head(5)

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,...,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,Unnamed: 21
0,2019,1,MQ,Envoy Air,SAV,"Savannah, GA: Savannah/Hilton Head International",65.0,15.0,3.41,0.71,...,6.56,1.0,1.0,601.0,180.0,29.0,129.0,0.0,263.0,NaN
1,2019,1,MQ,Envoy Air,SDF,"Louisville, KY: Louisville Muhammad Ali Intern...",61.0,18.0,2.70,1.01,...,5.37,1.0,0.0,890.0,180.0,36.0,383.0,0.0,291.0,NaN
2,2019,1,MQ,Envoy Air,SGF,"Springfield, MO: Springfield-Branson National",428.0,80.0,13.31,5.18,...,34.09,15.0,0.0,3954.0,705.0,213.0,982.0,0.0,2054.0,NaN
3,2019,1,MQ,Envoy Air,SHV,"Shreveport, LA: Shreveport Regional",174.0,28.0,5.97,1.17,...,9.72,0.0,0.0,1655.0,360.0,55.0,268.0,0.0,972.0,NaN
4,2019,1,MQ,Envoy Air,SJT,"San Angelo, TX: San Angelo Regional/Mathis Field",135.0,23.0,10.78,0.35,...,5.33,2.0,0.0,835.0,320.0,27.0,192.0,0.0,296.0,NaN


In [7]:
print(f"Total rows: {len(df)}")

Total rows: 42894


In [9]:
df.isnull().sum()

year                       0
 month                     0
carrier                    0
carrier_name               0
airport                    0
airport_name               0
arr_flights              106
arr_del15                282
carrier_ct               106
 weather_ct              106
nas_ct                   106
security_ct              106
late_aircraft_ct         106
arr_cancelled            106
arr_diverted             106
 arr_delay               106
 carrier_delay           106
weather_delay            106
nas_delay                106
security_delay           106
late_aircraft_delay      106
Unnamed: 21            42894
dtype: int64

In [10]:
df.carrier_name.unique()

array(['Envoy Air', 'Spirit Air Lines', 'PSA Airlines Inc.',
       'SkyWest Airlines Inc.', 'United Air Lines Inc.',
       'Southwest Airlines Co.', 'Mesa Airlines Inc.', 'Republic Airline',
       'Endeavor Air Inc.', 'American Airlines Inc.',
       'Alaska Airlines Inc.', 'JetBlue Airways', 'Delta Air Lines Inc.',
       'ExpressJet Airlines Inc.', 'Frontier Airlines Inc.',
       'Allegiant Air', 'Hawaiian Airlines Inc.',
       'ExpressJet Airlines LLC', 'Horizon Air'], dtype=object)

In [11]:
df.carrier.unique()

array(['MQ', 'NK', 'OH', 'OO', 'UA', 'WN', 'YV', 'YX', '9E', 'AA', 'AS',
       'B6', 'DL', 'EV', 'F9', 'G4', 'HA', 'QX'], dtype=object)